# Project Overview: Predicting the Strength of MCTS Variants in General Game Playing


**Introduction to Monte Carlo Tree Search (MCTS):**

Monte Carlo Tree Search (MCTS) is a widely used algorithm in game AI for decision-making in uncertain environments. It has gained prominence due to its effectiveness in a wide range of games, from Go to general board games. MCTS operates by simulating random playouts of potential game states and expanding the most promising branches of a game tree, balancing exploration of new strategies with the exploitation of known successful paths. However, MCTS comes in various variants, each tailored to specific types of games or problem spaces.

This project seeks to predict the relative performance of different MCTS variants in various board games. The competition goal is to analyze how MCTS configurations affect outcomes and understand which variants are most suitable for different game environments. We aim to build a model that accurately predicts the game outcomes based on a set of features describing the games and the specific MCTS agent configurations.

**Key Concepts in General Game Playing**

In the broader context of artificial intelligence research, general game-playing (GGP) refers to the development of AI agents that can play a variety of games without being specialized for any particular game. Unlike traditional AI models that are designed to master specific games (such as AlphaGo for Go), general game-playing agents aim to be adaptable across different game types without pre-existing knowledge. This project focuses on evaluating MCTS variants within such a general game-playing framework.

General Game Playing agents are designed to function without knowing the games they will play in advance. This contrasts with game-specific AI agents, which often use heuristics and human-designed strategies to excel in specific environments. GGP agents must instead rely on learning and exploration, making the performance of MCTS variants highly relevant to the success of these agents.


**MCTS variants can differ across several dimensions, including:**

* Selection Strategy: How the agent selects which game state to explore next. Popular strategies include Upper Confidence Bound (UCB1), Progressive History, and more.
* Exploration Constant: A parameter controlling the balance between exploring new strategies and exploiting known ones.
* Playout Policy: How the agent simulates the outcome of a game from a given state, which can range from random playouts to more sophisticated heuristics like Move-Average Sampling Techniques (MAST).
* Score Bounds: Some variants also use score bounds to limit or normalize the evaluation of game states.

Each of these aspects can have a significant impact on the agent's ability to perform in different game environments. The competition dataset includes various combinations of these features, and our task is to learn which configurations lead to the best performance under different game scenarios.


**Objective of the Project**

Our objective is to develop a machine learning model that can accurately predict the performance (game outcomes) of MCTS variants in general board games. To achieve this, we will analyze the dataset provided by the competition, which includes various game features and agent configurations. Through this analysis, we aim to uncover patterns that reveal how certain MCTS variants excel in specific game types.

**Research Significance**

Understanding the performance of MCTS variants in different contexts is crucial for advancing AI in general game playing. This project has implications for:
* Agent Selection: Choosing the most effective agent for a given game environment.
* Transfer Learning: Applying knowledge gained from one game to another, based on similarities in game structure.
* AI Explainability: Providing insights into why certain variants perform better, which can help design more transparent and interpretable AI systems.
* Generalization: Developing AI models that are not restricted by domain-specific knowledge and can excel across a broad range of games.


**Approach**

We will employ machine learning techniques such as Random Forest Regressors to model the relationship between game features and outcomes. By preprocessing the data to handle categorical variables (e.g., agent descriptions), applying techniques such as feature encoding and model evaluation, we aim to produce a predictive model that operates efficiently within the competition constraints. The dataset includes hundreds of features describing game rules, agents, and configurations, making feature selection and efficient model training critical.

Why Random Forest? This model is robust for handling a mix of categorical and numerical features. It doesn’t require heavy scaling of data and can model complex relationships, making it a good choice for this multi-dimensional data. Since we are predicting a continuous value (degree of advantage), Random Forest’s regression capabilities can help in modeling non-linear interactions between different features.

**Preprocessing Pipeline:**

* **Handling Missing Data**: Missing values in the dataset must be imputed. We use SimpleImputer to fill in missing values (mean/mode imputation), which is common for numerical and categorical data.
* **Encoding Categorical Features**: Since the MCTS agent description has various categorical components, we break them down (like selection, exploration_const, etc.). These features are encoded using Ordinal Encoding to convert them into numerical values so that they can be fed into machine learning algorithms.

In [ ]:
# Step 1: Load the data

import pandas as pd

# Path to the dataset
train_path = '/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv'

# Load the data using Pandas
train_df = pd.read_csv(train_path)

In [ ]:
# Step 2: Check the structure of the dataset

print("First few rows of the dataset:")
print(train_df.head())

# Check column names and data types
print("\nColumn names and data types:")
print(train_df.dtypes)

In [ ]:
# Step 3: Check for missing values

print("\nChecking for missing values:")
print(train_df.isnull().sum())

In [ ]:
# Step 4: Summary statistics of numeric columns

print("\nSummary of numeric columns:")
print(train_df.describe())

In [ ]:
# Step 5: Unique values in categorical columns

# Identify the object (string) columns for further processing
string_cols = train_df.select_dtypes(include=['object'])

print("\nUnique values in categorical columns:")
for col in string_cols.columns:
    print(f"Unique values in {col}: {train_df[col].nunique()}")

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Load the dataset
train_df = pd.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')

def split_agent_description(df, column_name):
    # Split the description into parts
    split_df = df[column_name].str.split('-', expand=True)
    split_df.columns = ['MCTS', 'selection', 'exploration_const', 'playout', 'score_bounds']
    
    # Combine the original dataframe with the new columns
    return pd.concat([df, split_df], axis=1).drop(columns=[column_name])


In [ ]:
agent1_df = split_agent_description(train_df, 'agent1')
agent2_df = split_agent_description(agent1_df, 'agent2')

In [ ]:
print(agent2_df.head())

In [ ]:
# Path to the concepts file
concepts_path = '/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv'

# Load the concepts file
concepts_df = pd.read_csv(concepts_path)

# Display the first few rows and column names
print("First few rows of the concepts dataset:")
print(concepts_df.head())

print("\nColumn names and data types:")
print(concepts_df.dtypes)


In [ ]:
# Print out columns and a sample to understand the content
print("\nColumns in concepts dataset:")
print(concepts_df.columns)

print("\nSample data from concepts dataset:")
print(concepts_df.head())

In [ ]:
# Example: If there's a column named 'importance' indicating feature importance
if 'importance' in concepts_df.columns:
    # Define a threshold for importance (this depends on your specific use case)
    threshold = 0.5  # Example threshold; adjust as needed

    # Select features with importance above the threshold
    important_features = concepts_df[concepts_df['importance'] > threshold]
    
    # Assuming 'feature_name' is the column with feature names
    selected_features = important_features['feature_name'].tolist()
    print("\nImportant features based on importance:")
    print(selected_features)

    # Filter the main dataset to include only selected features
    filtered_df = train_df[selected_features + ['target']]  # Include target if needed

    # Display the filtered DataFrame
    print("\nFiltered DataFrame with selected features:")
    print(filtered_df.head())
else:
    print("\nNo importance column found. Inspect the concepts file manually for feature importance.")

In [ ]:
# Path to the concepts file (ensure you have the correct path)
concepts_path = '/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv'

# Load the concepts file
concepts_df = pd.read_csv(concepts_path)

# Display the first few rows and column names
print("Step 1: First few rows of the concepts dataset:")
print(concepts_df.head())

print("\nStep 1: Column names and data types:")
print(concepts_df.dtypes)

# Print columns to understand the available data
print("\nStep 1: Columns in concepts dataset:")
print(concepts_df.columns)

In [ ]:
print("Available columns in train_df:")
print(train_df.columns.tolist())

In [ ]:
# Print all column names in train_df
print("Columns in train_df:")
print(train_df.columns.tolist())

In [ ]:
# Define the desired features
desired_features = ['NumPlayers', 'Simulation', 'Turns', 'GameOutcome']

# Filter the DataFrame to include only the desired features
filtered_df = train_df[desired_features]

print("\nStep 2: Filtered DataFrame with selected features:")
print(filtered_df.head())

In [ ]:
# Columns in train_df that are not in desired_features
missing_features = set(train_df.columns) - set(desired_features)
print("\nColumns not in desired_features but present in train_df:")
print(missing_features)

In [ ]:
# Ensure desired_features contains all the columns you need
desired_features = ['GameOutcome', 'Simulation', 'NumPlayers', 'Turns']

# Filter train_df to keep only the columns in desired_features
filtered_train_df = train_df[desired_features]

# Check the columns in the filtered DataFrame
print(filtered_train_df.columns)

In [ ]:
# Check for missing values
print(filtered_train_df.isnull().sum())

# Summary statistics
print(filtered_train_df.describe())

In [ ]:
# Check the column names in the DataFrame
print(filtered_train_df.columns)

In [ ]:
# Check if the original dataset has the 'agent1' and 'agent2' columns
print(train_df.columns)  # Replace 'train_df' with your original DataFrame name

In [ ]:
# Re-load the dataset if necessary
train_df = pd.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')

# Keep the relevant columns for further processing
filtered_train_df = train_df[['agent1', 'agent2', 'GameOutcome']]

In [ ]:
# Check the number of columns produced by splitting 'agent1'
split_agent1 = filtered_train_df['agent1'].str.split('-', expand=True)
print("Split agent1 columns:", split_agent1.shape[1])

# Check the number of columns produced by splitting 'agent2'
split_agent2 = filtered_train_df['agent2'].str.split('-', expand=True)
print("Split agent2 columns:", split_agent2.shape[1])

In [ ]:
# Inspect split results to understand the extra column
print("Extra column in split_agent1:")
print(split_agent1.head())

print("Extra column in split_agent2:")
print(split_agent2.head())

In [ ]:
# Check the DataFrame structure and the new columns
print(filtered_train_df.head())
print(filtered_train_df.columns)

In [ ]:
# Check for any missing values in the new columns
print(filtered_train_df[['selection1', 'exploration_const1', 'playout1', 'score_bounds1', 'selection2', 'exploration_const2', 'playout2', 'score_bounds2']].isnull().sum())

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Initialize OrdinalEncoder
encoder = OrdinalEncoder()

# List of categorical columns
categorical_columns = ['selection1', 'exploration_const1', 'playout1', 'score_bounds1', 'selection2', 'exploration_const2', 'playout2', 'score_bounds2']

# Apply encoding to the categorical columns
filtered_train_df.loc[:, categorical_columns] = encoder.fit_transform(filtered_train_df[categorical_columns])

# Verify the DataFrame after encoding
print(filtered_train_df.head())

In [ ]:
# Check for missing values in the entire DataFrame
print(filtered_train_df.isnull().sum())

In [ ]:
filtered_train_df['GameOutcome'] = filtered_train_df['GameOutcome'].fillna(0)  # Replace 0 with your desired value

In [ ]:
mean_value = filtered_train_df['GameOutcome'].mean()
filtered_train_df['GameOutcome'] = filtered_train_df['GameOutcome'].fillna(mean_value)

In [ ]:
median_value = filtered_train_df['GameOutcome'].median()
filtered_train_df['GameOutcome'] = filtered_train_df['GameOutcome'].fillna(median_value)

In [ ]:
mode_series = filtered_train_df['GameOutcome'].mode()
print("Mode series:", mode_series)

In [ ]:
if not mode_series.empty:
    mode_value = mode_series[0]
else:
    # Define a default value if mode is empty
    mode_value = 0  # or another value that fits your context

filtered_train_df['GameOutcome'] = filtered_train_df['GameOutcome'].fillna(mode_value)

In [ ]:
print(filtered_train_df['GameOutcome'].isnull().sum())

In [ ]:
default_value = 0  # or another meaningful value
filtered_train_df['GameOutcome'] = filtered_train_df['GameOutcome'].fillna(default_value)

In [ ]:
filtered_train_df = filtered_train_df.dropna(subset=['GameOutcome'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Define features and target
X = filtered_train_df.drop(['GameOutcome'], axis=1)
y = filtered_train_df['GameOutcome']

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
scores = cross_val_score(model, X, y, cv=5, scoring='r2')  # or any other metric you prefer

print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error

# Define the model
model = RandomForestRegressor(random_state=42)

# Define K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Define scoring function (mean squared error)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring=scorer)

# Print cross-validation results
print("Cross-Validation MSE Scores: ", -cv_scores)
print("Mean MSE: ", -cv_scores.mean())
print("Standard Deviation: ", cv_scores.std())

In [ ]:
print(filtered_train_df.shape)  # Should return (n_samples, n_features)
print(filtered_train_df.head())  # Display the first few rows of the dataframe